In [8]:
pip --version

pip 19.2.3 from /home/ubudubuntu/Documents/krzysztof/jupyter/venv/lib/python3.8/site-packages/pip (python 3.8)
Note: you may need to restart the kernel to use updated packages.


In [5]:
import sklearn.datasets

wine_data = sklearn.datasets.load_wine()
frame_wine_data = sklearn.datasets.load_wine(as_frame = True)

In [6]:
def print_data():
    global wine_data
    #print(wine_data)
    global frame_wine_data
    print(frame_wine_data)

print_data()

{'data':      alcohol  malic_acid   ash  alcalinity_of_ash  magnesium  total_phenols  \
0      14.23        1.71  2.43               15.6      127.0           2.80   
1      13.20        1.78  2.14               11.2      100.0           2.65   
2      13.16        2.36  2.67               18.6      101.0           2.80   
3      14.37        1.95  2.50               16.8      113.0           3.85   
4      13.24        2.59  2.87               21.0      118.0           2.80   
..       ...         ...   ...                ...        ...            ...   
173    13.71        5.65  2.45               20.5       95.0           1.68   
174    13.40        3.91  2.48               23.0      102.0           1.80   
175    13.27        4.28  2.26               20.0      120.0           1.59   
176    13.17        2.59  2.37               20.0      120.0           1.65   
177    14.13        4.10  2.74               24.5       96.0           2.05   

     flavanoids  nonflavanoid_phenols  pro

In [7]:
from sklearn.model_selection import train_test_split

def split_train_test(train_percentage):
    X = frame_wine_data['data']
    y = frame_wine_data['target']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=(1.0 - train_percentage))
    return X_train, y_train, X_test, y_test

X_train, y_train, X_test, y_test = split_train_test(0.6)

In [8]:
print(len(X_train))
print(len(X_test))
print(len(y_train))
print(len(y_test))

X_train.to_csv('X_train.csv')
X_test.to_csv('X_test.csv')
y_train.to_csv('y_train.csv')
y_test.to_csv('y_test.csv')

106
72
106
72


In [10]:
def q_analyse(dataset):
    print('Dataset length: {}, values: {}'.format(len(dataset), dataset.size))
    n_unique = dataset.nunique()
    #print('Unique values:')
    #print(n_unique)
    print('Num of unique values: {}'.format(n_unique.cumsum()[-1]))
    #skipNa default true
    mean_val = dataset.mean()
    print('Mean values:')
    print(mean_val)
    
    print('Null values per column:')
    nulls = len(dataset) - dataset.count()
    #print(nulls)
    print('Null values sum: {}'.format(nulls.cumsum()[-1]))
    
    print('Maximal values:')
    print(dataset.max())
    
    print('Minimal values:')
    print(dataset.min())
    
    print('Most common values:')
    print('for indexes:')
    print(dataset.mode(axis = 0))
    print('for columns:')
    print(dataset.mode(axis = 1))
    

print('Train dataset:')
q_analyse(X_train)
print()
print('Test dataset:')
q_analyse(X_test)

Train dataset:
Dataset length: 106, values: 1378
Num of unique values: 895
Mean values:
alcohol                          13.017264
malic_acid                        2.290755
ash                               2.372925
alcalinity_of_ash                19.411321
magnesium                       100.584906
total_phenols                     2.280755
flavanoids                        1.969623
nonflavanoid_phenols              0.366038
proanthocyanins                   1.509717
color_intensity                   5.134717
hue                               0.955434
od280/od315_of_diluted_wines      2.562170
proline                         766.254717
dtype: float64
Null values per column:
Null values sum: 0
Maximal values:
alcohol                           14.75
malic_acid                         5.04
ash                                3.22
alcalinity_of_ash                 30.00
magnesium                        162.00
total_phenols                      3.85
flavanoids                         3.75